## Generator 可以使用 next 來進行循環中的一步
文字上有點難解釋，直接來看範例就能了解什麼是 Generator!

### 撰寫一個 Generator，一次吐出 list 中的一個值

In [ ]:
def output_from_list_generator(your_list):
    for i in your_list:
        yield i 

In [ ]:
my_list = [1, 2, 3, 4, 5]

In [ ]:
gen = output_from_list_generator(my_list)

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

### 從上面的範例程式碼我們可以看到，當使用一次 next，generator 就會跑 for_loop 一次，因此得到 list 中的第一個值，當再使用一次後，for_loop 記得上次的循環，所以吐出第二個值。最後一次，因為 for loop 已經執行結束了，所以再使用 next 就會看到 StopIteration，無法在得到值

### 我們可以撰寫一個無限循環的 Generator，只要使用 While True 即可

In [ ]:
def inf_loop_generator(your_list):
    while True:
        for i in your_list:
            yield i

In [ ]:
gen = inf_loop_generator(my_list)

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

In [ ]:
print(next(gen))

### 上面的程式碼因為我們使用了 While True，所以 for loop 不會結束，只要 call next 就一定會跑一次循環，並返回值

## 雖然 Cifar-10 的資料可以全部讀進記憶體，但讓我們試著用 Generator，批次的把 Cifar 10 的資料取出來，一次取 32 張出來！

In [ ]:
def img_combine(img, ncols=8, size=1, path=False):
    from math import ceil #往上進位至整數
    import matplotlib.pyplot as plt
    import numpy as np
    nimg = len(img)
    nrows = int(ceil(nimg/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(ncols*size,nrows*size))
    if nrows == 0:
        return
    elif ncols == 1:
        for r, ax in zip(np.arange(nrows), axes):
            nth=r
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                
            ax.set_axis_off()
    elif nrows == 1:
        for c, ax in zip(np.arange(ncols), axes):
            nth=c
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
            ax.set_axis_off()
    else:
        for r, row in zip(np.arange(nrows), axes):
            for c, ax in zip(np.arange(ncols), row):
                nth=r*ncols+c
                if nth < nimg:
                    ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                ax.set_axis_off()
    plt.show()

In [ ]:
from keras.datasets import cifar10

In [ ]:
(x_train, x_test), (y_train, y_test) = cifar10.load_data()

In [ ]:
def cifar_generator(image_array, batch_size=32):
    while True:
        for indexs in range(0, len(image_array), batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = x_test[indexs: indexs+batch_size]
            yield images, labels

In [ ]:
cifar_gen = cifar_generator(x_train)

In [ ]:
images, labels = next(cifar_gen)

In [ ]:
img_combine(images)

In [ ]:
images, labels = next(cifar_gen)

In [ ]:
img_combine(images)

## 可以看到兩次的圖片並不一樣，這樣就可以開始訓練囉！

## 作業

請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras [官方文件中 fit_generator 的說明](https://keras.io/models/sequential/)

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
___________________________________________________________

In [2]:
# def cifar_generator(image_array, batch_size=32):
#     while True:
#         for indexs in range(0, len(image_array), batch_size):
#             images = x_train[indexs: indexs+batch_size]
#             labels = x_test[indexs: indexs+batch_size]
#             yield images, labels
from sklearn.utils import shuffle
def my_generator(x, y, batch_size):
    while True:
        for idx in range(0, len(x), batch_size): # 讓 idx 從 0 開始，一次增加 batch size。假設 batch_size=32, idx = 0, 32, 64, 96, ....
            batch_x, batch_y = x[idx:idx+batch_size], y[idx:idx+batch_size]
            yield batch_x, batch_y
        x, y = shuffle(x, y) # loop 結束後，將資料順序打亂再重新循環
train_generator = my_generator(x_train, y_train, batch_size) # 建立好我們寫好的 generator

In [3]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
390/390 [==============================] - 10s 26ms/step - loss: 1.7427 - acc: 0.3705 - val_loss: 1.3547 - val_acc: 0.5104
Epoch 2/10
390/390 [==============================] - 7s 18ms/step - loss: 1.3367 - acc: 0.5261 - val_loss: 1.2981 - val_acc: 0.5445
Epoch 3/10
390/390 [==============================] - 7s 18ms/step - loss: 1.1353 - acc: 0.6009 - val_loss: 1.1837 - val_acc: 0.5915
Epoch 4/10
390/390 [==============================] - 7s 18ms/step - loss: 1.0053 - acc: 0.6473 - val_loss: 0.9389 - val_acc: 0.6698
Epoch 5/10
390/390 [==============================] - 7s 18ms/step - loss: 0.9053 - acc: 0.6840 - val_loss: 0.7877 - val_acc: 0.7250
Epoch 6/10
390/390 [==============================] - 7s 18ms/step - loss: 0.8406 - acc: 0.7063 - val_loss: 0.7843 - val_acc: 0.7229
Epoch 7/10
390/390 [==============================] - 7s 18ms/step - loss: 0.7806 - acc: 0.7276 - val_loss:

In [12]:
import numpy as np
# print(np.asarray(next(train_generator)).shape())
print(next(train_generator))

(array([[[[0.8352941 , 0.85490197, 0.88235295],
         [0.8156863 , 0.84705883, 0.88235295],
         [0.8117647 , 0.85882354, 0.8901961 ],
         ...,
         [0.79607844, 0.85490197, 0.8901961 ],
         [0.81960785, 0.8627451 , 0.8784314 ],
         [0.8156863 , 0.8509804 , 0.88235295]],

        [[0.81960785, 0.8509804 , 0.88235295],
         [0.8117647 , 0.8509804 , 0.8862745 ],
         [0.8156863 , 0.85882354, 0.8980392 ],
         ...,
         [0.8901961 , 0.8980392 , 0.8862745 ],
         [0.9254902 , 0.9254902 , 0.9019608 ],
         [0.88235295, 0.88235295, 0.88235295]],

        [[0.8666667 , 0.9019608 , 0.92941177],
         [0.85490197, 0.8980392 , 0.93333334],
         [0.84705883, 0.8901961 , 0.9137255 ],
         ...,
         [0.90588236, 0.8980392 , 0.8862745 ],
         [0.9019608 , 0.8901961 , 0.88235295],
         [0.8862745 , 0.88235295, 0.87058824]],

        ...,

        [[0.69411767, 0.69803923, 0.7058824 ],
         [0.7019608 , 0.7058824 , 0.70980394